In [34]:
import numpy as np
import pandas as pd
from os import listdir
from os.path import join, isfile

# info tables
simtable = f'/data01/homes/dipiano/E4/irf_random/crab/sim/merged_simulator_data.dat'
simdata = pd.read_csv(infotable, sep=' ', header=0).sort_values(by=['seed'])
simdata.head()

,name,seed,start,stop,duration,source_ra,source_dec,point_ra,point_dec,offset,irf,fov,sim_time
60000,crab_00001,1,0,100,100,313.160025,23.341878,313.197506,22.894122,0.449081,North_z40_N_5h_LST,2.5,8.043759
60001,crab_00002,2,0,100,100,20.761826,-79.812129,30.555079,-80.334273,1.764702,North_z60_0.5h_LST,2.5,7.736644
60002,crab_00003,3,0,100,100,263.050678,-20.730728,263.711201,-21.511765,0.994811,North_z60_S_0.5h_LST,2.5,7.631279
60003,crab_00004,4,0,100,100,28.792668,-35.000006,28.821520,-35.011190,0.026146,North_z60_S_0.5h_LST,2.5,7.664158
60004,crab_00005,5,0,100,100,37.429610,-41.740501,38.148164,-41.630777,0.547721,North_z60_S_0.5h_LST,2.5,7.609625


In [35]:
import matplotlib.pyplot as plt
import astropy.units as u
from astropy import wcs

# extract heatmap from DL3 with configurable exposure
def extract_heatmap_from_table(data, smoothing, nbins, save=False, save_name=None, filter=True, 
                               trange=None, wcs=None):
    if filter and trange is not None:
        data = data[(data['TIME'] >= trange[0]) & (data['TIME'] <= trange[1])] 
    ra, dec = data['RA'].to_numpy(), data['DEC'].to_numpy()
    
    if wcs is not None:
        ra, dec = wcs.world_to_pixel(SkyCoord(ra, dec, unit='deg'))
    heatmap, xe, ye = np.histogram2d(ra, dec, bins=nbins)
    if smoothing != 0:
        heatmap = gaussian_filter(heatmap, sigma=smoothing)
    if save and save_name is not None:
        np.save(save_name, heatmap, allow_pickle=True, fix_imports=True)
    return heatmap.T

# plot heatmap with WCS
def plot_heatmap_wcs(heatmap, transform, title='heatmap', xlabel='x', ylabel='y', show=False, save=False, 
                     save_name=None, wcs=None, src=None, yflip=False):
    if wcs is not None:
        ax = plt.subplot(projection=wcs)
        ax.coords[0].set_format_unit(u.deg)
        ax.coords[1].set_format_unit(u.deg)
        ax.invert_xaxis()
        ax.scatter(src.ra.deg, src.dec.deg, marker='o', s=50, facecolor='none', edgecolor='k', 
                   transform=ax.get_transform(transform))
    else:
        ax = plt.subplot()
    if yflip:
        heatmap = np.flipud(heatmap)
    img = ax.imshow(heatmap, vmin=0, vmax=1, origin='lower')
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    plt.colorbar(img)
    if save and save_name is not None:
        plt.savefig(save_name)
    if show:
        plt.show()
    plt.close()
    return

In [36]:
from astropy.table import Table
from astroai.tools.utils import extract_heatmap_from_table
from astropy.wcs import WCS
from astropy.io import fits
from astropy.coordinates import SkyCoord

seed = 1
print(f"SEED={seed}")

row = simdata[simdata['seed']==seed]
row

SEED=1


,name,seed,start,stop,duration,source_ra,source_dec,point_ra,point_dec,offset,irf,fov,sim_time
60000,crab_00001,1,0,100,100,313.160025,23.341878,313.197506,22.894122,0.449081,North_z40_N_5h_LST,2.5,8.043759


In [37]:
src = SkyCoord(ra=row['source_ra'].values[0], dec=row['source_dec'].values[0], unit='deg', frame='icrs')
src.ra.deg, src.dec.deg

(313.16002476539614, 23.34187847081937)

In [42]:
from astroai.tools.utils import extract_heatmap

img = f"/data01/homes/dipiano/E4/irf_random/crab/sim/crab_00001.fits"
print(img)

with fits.open(img) as h:
    w = WCS(header=h[0].header)
    data = h[1].data
    


/data01/homes/dipiano/E4/irf_random/crab/sim/crab_00001.fits


FITS_rec([(          1,  2.02900642, -46.980927, 23.272585, 0.0582465 ,  0.37856153, -0.16391507, 1),
          (          2, 10.81418926, -47.02641 , 23.524553, 0.04292103,  0.63058823, -0.20530923, 1),
          (          3, 10.99342662, -47.134007, 23.42104 , 0.05583354,  0.5272614 , -0.30420431, 1),
          ...,
          (       5059, 99.94415909, -47.898937, 23.590948, 0.05682643,  0.70060265, -1.004825  , 2),
          (       5060, 99.99390763, -45.28595 , 22.555899, 0.04373148, -0.33104676,  1.4005214 , 2),
          (       5061, 99.99856299, -47.16383 , 23.570683, 0.09219255,  0.67697114, -0.33119473, 2)],
         dtype=(numpy.record, [('EVENT_ID', '>i4'), ('TIME', '>f8'), ('RA', '>f4'), ('DEC', '>f4'), ('ENERGY', '>f4'), ('DETX', '>f4'), ('DETY', '>f4'), ('MC_ID', '>i4')]))

In [ ]:
heatmap = Table.read(img, hdu=1).to_pandas()
heatmap = extract_heatmap_from_table(heatmap, smoothing=5, nbins=250, wcs=w)

plot_heatmap_wcs(heatmap, title=f"seed {seed}", wcs=w, src=src, xlabel='RA [deg]', ylabel='DEC [deg]',
                 show=True, save=False, save_name=None, transform='world')

In [ ]:
# binning in RA, DEC
# converti solo SRC_RA, SRC_DEC
# plt in pixel la heatmap e la SRC